In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.metrics import roc_auc_score, confusion_matrix, RocCurveDisplay

In [30]:
df = pd.read_csv("../Oblig1/wdbc.data", header=None)
df["target"] = np.where(df[1] == "M", 1, 0)
df = df.drop([0,1], axis=1)

Først leser vi inn datasetet og ser hvilke verdier vi h. I og konvertere verdiene i target kollonen til 1 for M og 0 for B, slik at disse er på et format som kan brukes senere, vi legger også disse verdiene i en ny kollone slik at vi kan bruke disse  Vi tar også og droppe kollonen for ID-veriden og den gamle kollonen med target verdiene



ref for np.where: https://www.geeksforgeeks.org/numpy/numpy-where-in-python/

In [36]:
def split_dataset(data):
    train_val = data.groupby("target", group_keys=False)[data.columns].sample(frac=0.8)
    test = data.drop(train_val.index)
    train = train_val.groupby("target", group_keys=False)[train_val.columns].sample(frac=0.75)
    validation = train_val.drop(train.index)    
    return [train, validation, test]

def check_split_dataset(data):
    print(data[0]["target"].value_counts()/data[0].shape[0])
    print(data[1]["target"].value_counts()/data[1].shape[0])
    print(data[2]["target"].value_counts()/data[2].shape[0])

def split_targets_and_features(data):
    targets = data.target
    features = data.drop("target", axis=1)
    return [targets, features]

def create_set_of_targets_and_features(data):
    train = split_targets_and_features(data[0])
    validation = split_targets_and_features(data[1])
    test = split_targets_and_features(data[2])

    return [train, validation, test]

def create_full_training_dataset(input):
    data = split_dataset(input)
    dataset = create_set_of_targets_and_features(data)

    return dataset
        
def evaluate_model(classifier, features, targets):
    predictions = classifier.predict(features)
    accuracy = accuracy_score(targets, predictions)
    precision = precision_score(targets, predictions)
    recall = recall_score(targets, predictions)
    f1 = f1_score(targets, predictions)
    roc_auc = roc_auc_score(targets, predictions)

    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1: {f1:.2f}')
    print(f'ROCAUC: {roc_auc:.2f}')

def validate(classifier, train, validation):
    print('Training results:')
    evaluate_model(classifier, train[1], train[0])
    print('Validation results:')
    evaluate_model(classifier, validation[1], validation[0])



In [57]:
dataset = create_full_training_dataset(df)
#data = split_dataset(df)
#check_split_dataset(data)

dt1 = DecisionTreeClassifier()
dt1.fit(dataset[0][1], dataset[0][0])

validate(dt1, dataset[0], dataset[1])



Training results:
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1: 1.00
ROCAUC: 1.00
Validation results:
Accuracy: 0.93
Precision: 0.93
Recall: 0.88
F1: 0.90
ROCAUC: 0.92


In [2]:
df2 = pd.DataFrame({"test": [1,2,3], "test2": [4,5,6]})
df2.head()


,test,test2
0,1,4
1,2,5
2,3,6


In [3]:
row = {"test": 8, "test2": 9}
df2.loc[len(df2)]= row
df2.head()

,test,test2
0,1,4
1,2,5
2,3,6
3,8,9
